# Laboratorio 2
## Universidad del Valle de Guatemala <br> Facultas de Ingeniería
#### Departamento de Ciencias de la Computación <br> Deep Learning y Sistemas Inteligentes - Sección 20
#### Grupo 12  
Cristian Laynez, Jeyner Arango

### Objetivo de la Red

### Implementación de Redes

In [1]:
# Paquetes a utilizar
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import layers, regularizers

# Cargar la base de datos
dataset = pd.read_csv('movie_statistic_dataset.csv')

# eliminar las filas con valores faltantes en la columna de destino
dataset = dataset.dropna(subset=['Worldwide gross $'])

# Indexar 'movie_title'
dataset.set_index('movie_title', inplace=True)

#runtime_minutes
dataset['runtime_minutes'] = dataset['runtime_minutes'].astype(int)

#director_name
#Es mejor convertir los directores faltantes a NaN (No es un número)
dataset['director_name'].replace('-', np.nan, inplace=True)

#production_date: AAAA-MM-DD
#Como el campo representa una fecha, es mejor transformarlo en características separadas de
#año, mes y día. De esta manera, la red neuronal puede capturar mejor los patrones temporales.
dataset['production_year'] = pd.to_datetime(dataset['production_date']).dt.year
dataset['production_month'] = pd.to_datetime(dataset['production_date']).dt.month
dataset['production_day'] = pd.to_datetime(dataset['production_date']).dt.day

# botar la columnar original 'production_date' 
dataset.drop(columns=['production_date'], inplace=True)

#genres: varios géneros separados por comas
#Podemos usar la codificación one-hot para convertir 
#los géneros en columnas binarias separadas para cada género.
# Convertir genres a columnas binarias utilizando one-hot encoding
dataset['genres'].replace(r'\N', '', inplace=True)
genres_list = dataset['genres'].str.get_dummies(sep=',')
dataset = pd.concat([dataset, genres_list], axis=1)

# botar la columna original 'genres'
dataset.drop(columns=['genres'], inplace=True)

#director_professions: 
#Es mejor convertir los profesiones faltantes a NaN (No es un número)
dataset['director_professions'].replace('-', np.nan, inplace=True)
#Varias profesiones separadas por coma
#De manera similar a los géneros, podemos usar la codificación one-hot para 
#convertir las profesiones de director en columnas binarias separadas para cada profesión.
professions_list = dataset['director_professions'].str.get_dummies(sep=',')
dataset = pd.concat([dataset, professions_list], axis=1)

# botar la columna original 'director_professions'
dataset.drop(columns=['director_professions'], inplace=True)

#director:birthYear: valores faltantes como '-'
#Es mejor convertir los años de nacimiento faltantes a NaN (No es un número) para que
#se manejen correctamente durante el procesamiento de datos.
dataset['director_birthYear'].replace(r'\N', '-1', inplace=True)
dataset['director_birthYear'].replace('-', -1, inplace=True)
dataset['director_birthYear'] = dataset['director_birthYear'].astype(int)

# director:deathYear: valores faltantes como '-' y 'alive' si no está muerto
#Podemos convertir los valores 'vivos' a NaN y reemplazar el '-' con NaN también.
# Convertir 'alive' a NaN 
dataset['director_deathYear'].replace('alive', -1, inplace=True)
# Convertir '-' a NaN 
dataset['director_deathYear'].replace('-', -1, inplace=True)
dataset['director_deathYear'] = dataset['director_deathYear'].astype(int)

# director_name
# Por el momento boto el nombre del director 
dataset.drop(columns=['director_name'], inplace=True)

2023-08-05 14:47:06.106800: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-05 14:47:06.591916: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-05 14:47:06.595011: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-05 14:47:08.349905: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
dataset.columns

Index(['runtime_minutes', 'director_birthYear', 'director_deathYear',
       'movie_averageRating', 'movie_numerOfVotes', 'approval_Index',
       'Production budget $', 'Domestic gross $', 'Worldwide gross $',
       'production_year', 'production_month', 'production_day', 'Action',
       'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary',
       'Drama', 'Family', 'Fantasy', 'Film-Noir', 'History', 'Horror', 'Music',
       'Musical', 'Mystery', 'News', 'Romance', 'Sci-Fi', 'Sport', 'Thriller',
       'War', 'Western', 'actor', 'actress', 'animation_department',
       'art_department', 'art_director', 'assistant_director',
       'camera_department', 'casting_department', 'casting_director',
       'cinematographer', 'composer', 'costume_designer', 'director', 'editor',
       'editorial_department', 'executive', 'location_management',
       'make_up_department', 'miscellaneous', 'music_artist',
       'music_department', 'producer', 'production_designer',
   

In [3]:
dataset.dtypes.values

array([dtype('int64'), dtype('int64'), dtype('int64'), dtype('float64'),
       dtype('float64'), dtype('float64'), dtype('int64'), dtype('int64'),
       dtype('int64'), dtype('int64'), dtype('int64'), dtype('int64'),
       dtype('int64'), dtype('int64'), dtype('int64'), dtype('int64'),
       dtype('int64'), dtype('int64'), dtype('int64'), dtype('int64'),
       dtype('int64'), dtype('int64'), dtype('int64'), dtype('int64'),
       dtype('int64'), dtype('int64'), dtype('int64'), dtype('int64'),
       dtype('int64'), dtype('int64'), dtype('int64'), dtype('int64'),
       dtype('int64'), dtype('int64'), dtype('int64'), dtype('int64'),
       dtype('int64'), dtype('int64'), dtype('int64'), dtype('int64'),
       dtype('int64'), dtype('int64'), dtype('int64'), dtype('int64'),
       dtype('int64'), dtype('int64'), dtype('int64'), dtype('int64'),
       dtype('int64'), dtype('int64'), dtype('int64'), dtype('int64'),
       dtype('int64'), dtype('int64'), dtype('int64'), dtype('int64'),


In [4]:
dataset.iloc[0,:]

runtime_minutes                 192.0
director_birthYear             1954.0
director_deathYear               -1.0
movie_averageRating               7.8
movie_numerOfVotes           277543.0
                               ...   
special_effects                   0.0
stunts                            0.0
transportation_department         0.0
visual_effects                    0.0
writer                            1.0
Name: Avatar: The Way of Water, Length: 67, dtype: float64

In [5]:
# Extraer características y meta 
y = dataset['Worldwide gross $']
X = dataset.drop(columns=['Worldwide gross $'])

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Identificar columnas numericas para estandarizar (excluyendo no-numericas)
numeric_columns = X_train.select_dtypes(include=['float64', 'int64','int32']).columns

# Standardize the numeric input features
scaler = StandardScaler()
X_train[numeric_columns] = scaler.fit_transform(X_train[numeric_columns])
X_test[numeric_columns] = scaler.transform(X_test[numeric_columns])

In [6]:
# 1. Red Neuronal con Activación Sigmoidal y Regularización L1:
model_1 = tf.keras.Sequential([
    layers.Dense(64, activation='sigmoid', input_shape=(X_train.shape[1],)),
    layers.Dense(32, activation='sigmoid'),
    layers.Dense(16, activation='sigmoid'),
    layers.Dense(1)
])

# Agregar regularización L1 a todas las capas ocultas
model_1.add(layers.Dense(16, activation='sigmoid', kernel_regularizer=regularizers.l1(0.01)))

# Compilar el modelo
model_1.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Entrenar el modelo
model_1.fit(X_train, y_train, epochs=20, batch_size=30, validation_data=(X_test, y_test))

Epoch 1/20
117/117 [==============================] - 2s 5ms/step - loss: 51541780805451776.0000 - mae: 111766544.0000 - val_loss: 61200131462004736.0000 - val_mae: 127529608.0000
Epoch 2/20
117/117 [==============================] - 0s 3ms/step - loss: 51541785100419072.0000 - mae: 111766568.0000 - val_loss: 61200131462004736.0000 - val_mae: 127529608.0000
Epoch 3/20
117/117 [==============================] - 0s 3ms/step - loss: 51541776510484480.0000 - mae: 111766504.0000 - val_loss: 61200131462004736.0000 - val_mae: 127529608.0000
Epoch 4/20
117/117 [==============================] - 0s 3ms/step - loss: 51541772215517184.0000 - mae: 111766536.0000 - val_loss: 61200131462004736.0000 - val_mae: 127529608.0000
Epoch 5/20
117/117 [==============================] - 0s 3ms/step - loss: 51541789395386368.0000 - mae: 111766576.0000 - val_loss: 61200131462004736.0000 - val_mae: 127529608.0000
Epoch 6/20
117/117 [==============================] - 0s 3ms/step - loss: 51541776510484480.0000 - m

In [7]:
# 2. Red Neuronal con Activación ReLU y Regularización Dropout:
model_2 = tf.keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dropout(0.2),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(1)
])

# Compilar el modelo
model_2.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Entrenar el modelo
model_2.fit(X_train, y_train, epochs=20, batch_size=30, validation_data=(X_test, y_test))

Epoch 1/20
117/117 [==============================] - 1s 4ms/step - loss: 51541720675909632.0000 - mae: 111766400.0000 - val_loss: 61199860879065088.0000 - val_mae: 127529064.0000
Epoch 2/20
117/117 [==============================] - 0s 3ms/step - loss: 51539607552000000.0000 - mae: 111762616.0000 - val_loss: 61192520779956224.0000 - val_mae: 127517408.0000
Epoch 3/20
117/117 [==============================] - 0s 3ms/step - loss: 51523020388302848.0000 - mae: 111735136.0000 - val_loss: 61154024988082176.0000 - val_mae: 127459208.0000
Epoch 4/20
117/117 [==============================] - 0s 3ms/step - loss: 51461898708713472.0000 - mae: 111641480.0000 - val_loss: 61041161837477888.0000 - val_mae: 127290632.0000
Epoch 5/20
117/117 [==============================] - 0s 4ms/step - loss: 51313090976808960.0000 - mae: 111409312.0000 - val_loss: 60783579763834880.0000 - val_mae: 126908848.0000
Epoch 6/20
117/117 [==============================] - 0s 3ms/step - loss: 51016450470576128.0000 - m

In [8]:
# 3. Red Neuronal con Activación Tangente Hiperbólica (Tanh) y Regularización L2:
model_3 = tf.keras.Sequential([
    layers.Dense(256, activation='tanh', kernel_regularizer=regularizers.l2(0.01), input_shape=(X_train.shape[1],)),
    layers.Dense(128, activation='tanh', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dense(64, activation='tanh', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dense(32, activation='tanh', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dense(1)
])

# Compilar el modelo
model_3.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Entrenar el modelo
model_3.fit(X_train, y_train, epochs=20, batch_size=30, validation_data=(X_test, y_test))

Epoch 1/20
117/117 [==============================] - 2s 8ms/step - loss: 51541776510484480.0000 - mae: 111766520.0000 - val_loss: 61200131462004736.0000 - val_mae: 127529576.0000
Epoch 2/20
117/117 [==============================] - 0s 4ms/step - loss: 51541785100419072.0000 - mae: 111766520.0000 - val_loss: 61200131462004736.0000 - val_mae: 127529568.0000
Epoch 3/20
117/117 [==============================] - 0s 4ms/step - loss: 51541759330615296.0000 - mae: 111766536.0000 - val_loss: 61200131462004736.0000 - val_mae: 127529568.0000
Epoch 4/20
117/117 [==============================] - 0s 4ms/step - loss: 51541780805451776.0000 - mae: 111766536.0000 - val_loss: 61200131462004736.0000 - val_mae: 127529568.0000
Epoch 5/20
117/117 [==============================] - 0s 4ms/step - loss: 51541780805451776.0000 - mae: 111766520.0000 - val_loss: 61200127167037440.0000 - val_mae: 127529560.0000
Epoch 6/20
117/117 [==============================] - 0s 4ms/step - loss: 51541780805451776.0000 - m

#### Acontinuacion se eliminaran unas cuantas variables y posteriormente se correran

Como se podra apreciar se esta analizando la ganancia bruta a nivel mundial. Hay unas variables que no aportan mucho para analizar dicho objetivo.

In [9]:
# Llevar a cabo una copia de seguridad
data_frame = dataset.copy()
data_frame

,runtime_minutes,director_birthYear,director_deathYear,movie_averageRating,movie_numerOfVotes,approval_Index,Production budget $,Domestic gross $,Worldwide gross $,production_year,...,production_designer,production_manager,script_department,sound_department,soundtrack,special_effects,stunts,transportation_department,visual_effects,writer
movie_title,,,,,,,,,,,,,,,,,,,,,
Avatar: The Way of Water,192,1954,-1,7.8,277543.0,7.061101,460000000,667830256,2265935552,2022,...,0,0,0,0,0,0,0,0,0,1
Avengers: Endgame,181,-1,-1,8.4,1143642.0,8.489533,400000000,858373000,2794731755,2019,...,0,0,0,0,0,0,0,0,0,0
Pirates of the Caribbean: On Stranger Tides,137,1960,-1,6.6,533763.0,6.272064,379000000,241071802,1045713802,2011,...,0,0,0,0,0,0,0,0,0,0
Avengers: Age of Ultron,141,1964,-1,7.3,870573.0,7.214013,365000000,459005868,1395316979,2015,...,0,0,0,0,0,0,0,0,0,1
Avengers: Infinity War,149,-1,-1,8.4,1091968.0,8.460958,300000000,678815482,2048359754,2018,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Super Size Me,100,1970,-1,7.2,110078.0,6.017902,65000,11529368,22233808,2004,...,0,0,0,0,0,0,0,0,0,1
The Brothers McMullen,98,1968,-1,6.6,7986.0,4.231464,50000,10426506,10426506,1995,...,0,0,0,0,0,0,0,0,0,1
Gabriela,93,1973,-1,4.9,1593.0,2.526405,50000,2335352,2335352,2001,...,0,0,0,0,0,0,0,0,0,1


In [10]:
# Eliminación de algunas variables


### Composición y Resultados Obtenidos

#### Resultados de los primeros modelos utilizando todas las variables:

| Modelo Numero | mae | val_mae | loss | val_loss |
| --- | --- | --- | --- | --- |
| 1 |  |  |  |  |
| 2 |  |  |  |  |
| 3 |  |  |  |  |

#### Resultados de los modelso sin utilizar todas las variables:

| Modelo Numero | mae | val_mae | loss | val_loss |
| --- | --- | --- | --- | --- |
| 1 |  |  |  |  |
| 2 |  |  |  |  |
| 3 |  |  |  |  |

### Diferencia de rendimiento  conceptuales en la composición y resultados obtenidos en cada red neural.

...

### Red Neuronal Optima

...